In [0]:
%sql 

use postg;

drop table if exists emp_attendance;
create table emp_attendance
(
	employee 	varchar(10),
	dates 		date,
	status 		varchar(20)
);
insert into emp_attendance values('A1', '2024-01-01', 'PRESENT');
insert into emp_attendance values('A1', '2024-01-02', 'PRESENT');
insert into emp_attendance values('A1', '2024-01-03', 'PRESENT');
insert into emp_attendance values('A1', '2024-01-04', 'ABSENT');
insert into emp_attendance values('A1', '2024-01-05', 'PRESENT');
insert into emp_attendance values('A1', '2024-01-06', 'PRESENT');
insert into emp_attendance values('A1', '2024-01-07', 'ABSENT');
insert into emp_attendance values('A1', '2024-01-08', 'ABSENT');
insert into emp_attendance values('A1', '2024-01-09', 'ABSENT');
insert into emp_attendance values('A1', '2024-01-10', 'PRESENT');
insert into emp_attendance values('A2', '2024-01-06', 'PRESENT');
insert into emp_attendance values('A2', '2024-01-07', 'PRESENT');
insert into emp_attendance values('A2', '2024-01-08', 'ABSENT');
insert into emp_attendance values('A2', '2024-01-09', 'PRESENT');
insert into emp_attendance values('A2', '2024-01-10', 'ABSENT');

--SELECT * from emp_attendance;

In [0]:
%sql

select * from postg.emp_attendance order by employee,dates

In [0]:
%sql
with cte as (
SELECT *, row_number() over(partition by employee order by employee,dates) as rn  from emp_attendance
),
present as (
select *, row_number() over(partition by employee order by employee,dates) as prn,
(rn -  row_number() over(partition by employee order by employee,dates)) as flag
from cte where status='PRESENT' and employee='A1'
),
absent as (
select *, row_number() over(partition by employee order by employee,dates) as prn,
(rn -  row_number() over(partition by employee order by employee,dates)) as flag
from cte where status='ABSENT' and employee='A1'
),
present_date_seq as (
select distinct employee,
first_value(dates) over(partition by employee,flag order by employee,dates ) as first_date,
last_value(dates) over(partition by employee,flag order by employee,dates range between unbounded preceding and unbounded following ) as last_date,
status 
from present 
),
absent_date_seq as (
select distinct employee,
first_value(dates) over(partition by employee,flag order by employee,dates ) as first_date,
last_value(dates) over(partition by employee,flag order by employee,dates range between unbounded preceding and unbounded following ) as last_date,
status 
from absent 
)
select * from present_date_seq
union
select * from absent_date_seq
order by employee,first_date
